## Driver Drowsiness Detection

# New Section

### Phase - 3: Integrate the trained model (phase-1) with face detection (phase-2) to run the detection on the videos 

#### Import the required libs

In [1]:
import cv2  
import dlib
import numpy as np
from imutils import face_utils
from tensorflow import keras

#### Method to directly load the model

In [2]:
def load_model(weights_path='/content/phaseI_model'):
    model = keras.models.load_model(weights_path)
    model.summary()
    return model

#### Method to detect the face and eyes

In [3]:
!wget   http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 # DOWNLOAD LINK

!bunzip2 /content/shape_predictor_68_face_landmarks.dat.bz2

datFile =  "/content/shape_predictor_68_face_landmarks.dat"

--2022-12-05 16:48:17--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.1’

shape_predictor_68_ 100%[===================>]  61.07M  31.1MB/s    in 2.0s    

2022-12-05 16:48:19 (31.1 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.1’ saved [64040097/64040097]



In [4]:
def detect_face_eye(img, face_model_path= (datFile)):
    # Converting the recorded image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Applying filter to remove impurities
    gray = cv2.bilateralFilter(gray, 5, 1, 1)

    # Eye landmarks
    (L_start, L_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (R_start, R_end) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

    # Initializing the Models for Landmark and face Detection
    detector = dlib.get_frontal_face_detector()
    landmark_predict = dlib.shape_predictor(face_model_path)

    # Detecting the faces
    faces = detector(gray)

    # Detecting eyes for each face
    faces_coors = []
    eyes_per_face = []
    for face in faces:
        # Converting faces rectangle to coordinates
        faces_coors.append([(face.left(), face.top()), (face.right(), face.bottom())])

        shape = landmark_predict(gray, face)
        # Converting the shape class directly to a list of (x,y) coordinates
        shape = face_utils.shape_to_np(shape)

        # Parsing the landmarks list to extract lefteye and righteye landmarks
        lefteye = np.array(shape[L_start: L_end])
        x11 = lefteye[:, 0].min()
        y11 = lefteye[:, 1].min()
        x12 = lefteye[:, 0].max()
        y12 = lefteye[:, 1].max()

        righteye = np.array(shape[R_start: R_end])
        x21 = righteye[:, 0].min()
        y21 = righteye[:, 1].min()
        x22 = righteye[:, 0].max()
        y22 = righteye[:, 1].max()

        eyes_per_face.append([[(x11, y11), (x12, y12)], [(x21, y21), (x22, y22)]])
    return faces_coors, eyes_per_face

#### Method to plot the face and eyes with addional variable which tells whether the eyes are closed or not

In [5]:
def plot_face_eye(img, faces, eyes_per_face, eyes_open):
    # colors
    red = (0, 0, 255)
    blue = (255, 0, 0)
    green = (0, 255, 0)

    # Line thickness of 2 px
    thickness = 2

    # Drawing the rectangles
    img_res = img
    for i, f in enumerate(faces):
        start_f, end_f = f
        img_res = cv2.rectangle(img_res, start_f, end_f, blue, thickness)

        for e in eyes_per_face[i]:
            start_e, end_e = e
            if eyes_open is True:
                color = green
            else:
                color = red
            img_res = cv2.rectangle(img_res, start_e, end_e, color, thickness)
    return img_res

#### Method to predict the whether the eyes are closed or not, using the model 

In [6]:
def predict(img, base_eye_loc, model):
    (x11, y11), (x12, y12) = base_eye_loc
    crop_y_percent = (y12 - y11)
    crop_x_percent = (x12 - x11)
    crop_img = img[int(y11 - crop_y_percent):int(y12 + crop_y_percent),
               int(x11 - crop_x_percent):int(x12 + crop_x_percent)]
    crop_img = cv2.resize(crop_img, (300, 300), interpolation=cv2.INTER_LINEAR)
    images_list = [np.array(crop_img)]
    x = np.asarray(images_list)
    pr_mask = model.predict(x)
    out = pr_mask.argmax(axis=1)
    return out[0]

#### Method to load the video from the disk and analyze the video frame by frame, detection is done on each frame

In [7]:
def capture_and_analyze_video(video_path='sample', video_ext='.mp4', wait_time_in_secs=1):
    # loop over frames from the video file stream
    # results = []
    video = cv2.VideoCapture(video_path + video_ext)
    (grabbed, frame) = video.read()
    closed_eyes_count = 0
    frame_width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = video.get(cv2.CAP_PROP_FPS)
    fps = 10
    size = (int(frame_width), int(frame_height))
    fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
    video_writer = cv2.VideoWriter(video_path + "_out" + video_ext,
                                   fourcc,
                                   fps,
                                   size)
    model = load_model()
    eyes_closed_count = 0
    eyes_open_now = [0, 0]
    
    while grabbed:
        (grabbed, frame) = video.read()
        if grabbed is True:
            output = frame.copy()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces_coors, eyes_per_face = detect_face_eye(frame)

            for item in eyes_per_face:
                eyes_open_now = [predict(output, item[0], model), predict(output, item[1], model)]

            eyes_open = False
            if eyes_open_now[0] == 0 and eyes_open_now[1] == 0:
                eyes_open = False
                print('Eyes are closed')
            if eyes_open_now[0] == 1 and eyes_open_now[1] == 1:
                eyes_open = True
                print('Eyes are open')

            plotted_img = plot_face_eye(output, faces_coors, eyes_per_face, eyes_open)

            if eyes_open:
                eyes_closed_count = 0
            else:
                eyes_closed_count += 1

            face_x1 = faces_coors[0][0][0]
            face_y1 = faces_coors[0][0][1]

            if eyes_closed_count >= fps:
                font = cv2.FONT_HERSHEY_PLAIN
                bottomLeftCornerOfText = (face_x1, face_y1)
                fontScale = 3
                fontColor = (0, 0, 255)
                thickness = 3
                lineType = 1

                cv2.putText(plotted_img,
                            'Alert!',
                            bottomLeftCornerOfText,
                            font,
                            fontScale,
                            fontColor,
                            thickness,
                            lineType)

            video_writer.write(plotted_img)

    video_writer.release()
    video.release()
    cv2.destroyAllWindows()

### Run the main method to test the model on video

In [8]:
capture_and_analyze_video('/content/sample_vijay_1')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 300, 300, 10)      760       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 150, 150, 10)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 150, 150, 10)      2510      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 75, 75, 10)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 56250)             0         
                                                                 
 dense_2 (Dense)             (None, 20)               

#### Check the output video file